In [1]:
import cv2
import os
import numpy as np
import keras
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
#from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
modelName = "M_CNN_M1_25-02-18"
img_rows, img_cols = 176, 176
#input_shape = (224, 224, 3)
input_shape=(48, 48, 3)
categorical_var = 0
image_datagen_var = 1
cloud_var = 1
PATH = 'data/'
batch_size = 64

In [3]:


DATA_PATH = os.path.join(PATH, 'Training')
data_dir_list =np.sort(os.listdir(DATA_PATH))
data_dir_list

array(['.DS_Store', 'glioma_tumor', 'meningioma_tumor', 'no_tumor',
       'pituitary_tumor'], dtype='<U16')

In [4]:
DATA_PATH_Test = os.path.join(PATH, 'Testing')
data_dir_list_Test =np.sort(os.listdir(DATA_PATH_Test))
data_dir_list_Test

array(['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor'],
      dtype='<U16')

In [5]:
data_gen = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.5, 
    zoom_range=0.4, 
    rescale=1./255,
    vertical_flip=True, 
    validation_split=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True) 

In [6]:
train_generator = data_gen.flow_from_directory(
        DATA_PATH,
        target_size=(img_rows, img_cols), 
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb', 
        shuffle=True,   
        save_format='png', 
        subset="training")

Found 2297 images belonging to 4 classes.


In [7]:
train_generator.samples 

2297

In [8]:
train_generator.class_indices

{'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}

In [9]:
test_generator = data_gen.flow_from_directory(
        DATA_PATH_Test,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb', 
        shuffle=True, 
        seed=None,  
        save_format='png',
        subset="validation")

Found 78 images belonging to 4 classes.


In [10]:
prior = keras.applications.VGG16(
    include_top=False, 
    weights='imagenet',
    input_shape= input_shape
)
model = Sequential()
model.add(prior)
model.add(Flatten())
model.add(Dense(128, activation='relu', name='Dense_Intermediate'))
model.add(Dropout(0.1, name='Dropout_Regularization'))
model.add(Dense(4, activation='sigmoid', name='Output'))

In [11]:
#for cnn_block_layer in model.layers[0].layers:
#    cnn_block_layer.trainable = False
#model.layers[0].trainable = False

In [12]:
from keras.optimizers import RMSprop
model.compile(
    optimizer=RMSprop(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.fit(
    train_generator,
    #steps_per_epoch=len(train_generator.filenames) // batch_size,
    epochs=20,
    validation_data=test_generator,
    #validation_steps=len(train_generator.filenames) // batch_size,
    #class_weight=class_weights,
    #callbacks=[
     #   EarlyStopping(patience=3, restore_best_weights=True),
      #  ReduceLROnPlateau(patience=2)
    #]
)


Epoch 1/20


InvalidArgumentError:  Incompatible shapes: [64] vs. [1600]
	 [[node Equal (defined at <ipython-input-13-8d1897074272>:3) ]] [Op:__inference_train_function_2439]

Function call stack:
train_function
